# Item Collaborative Filtering
* This notebook implements item-based collaborative filtering
* Prediction is $\tilde r_{ij} = \dfrac{\sum_{k \in N(j)} w_{kj}^{\lambda_w}r_{ik}^{\lambda_r}}{\sum_{k \in N(j)} w_{kj}^{\lambda_w} + \lambda}$ for item-based collaborative filtering
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{kj}$ is the similarity between items $j$ and $k$
* $N(j)$ is the largest $K$ sorted by $w_{kj}$
* $\lambda_w, \lambda_r, \lambda$ are regularization parameters

In [1]:
import LinearAlgebra: norm
import Setfield: @set
import SparseArrays: sparse
import NBInclude: @nbinclude
@nbinclude("Alpha.ipynb");

## Determine the neighborhoods for each user and item

In [ ]:
function get_correlation_matrix_outdir(residual_alphas, name = name)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(residual_alphas))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @info "generating similarity matrix for $residual_alphas"
    training = RatingsDataset64(get_residuals("training", residual_alphas))
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        maximum(training.item),
    )
    S = zeros(eltype(R), maximum(training.item), maximum(training.item))

    norms = map(norm, eachslice(R, dims = 2))
    norms[norms.==0] .= 1 # prevent division by 0
    @tprogress Threads.@threads for i = 1:size(S)[1]
        S[:, i] = vec(R[:, i]' * R) ./ norms ./ norms[i]
    end

    write_params(Dict("S" => S), outdir)
    outdir
end;

In [3]:
function read_similarity_matrix(outdir)
    read_params(outdir)["S"]
end;

In [4]:
function get_abs_neighborhood(item, S, K)
    weights = S[:, item]
    # ensure that the neighborhood for an item does not include itself
    weights[item] = Inf
    K = Int(min(K, length(weights) - 1))
    order = partialsortperm(abs.(weights), 2:K+1, rev = true)
    order, weights[order]
end;

In [5]:
isnonzero(x) = !isapprox(x, 0.0, atol = eps(Float32))

# evaluate a quartic polynomial at x
function polynomial_transform(x, λ)
    y = abs(x)
    w = λ[1] + λ[2] * y + λ[3] * y ^ 2 + λ[4] * y^3 + λ[5] * y^4
    sign(x) * w
end

# each prediction is just the weighted sum of all items in the neighborhood
# we apply regularization terms to decay the weights, ratings, and final prediction
function make_prediction(item, users, R, get_neighborhood, λ)
    if item > size(R)[2]
        # the item was not in our training set; we have no information
        return zeros(length(item))
    end
    items, weights = get_neighborhood(item)
    weights = polynomial_transform.(weights, tuple(λ[1:5]))
    predictions = zeros(eltype(weights), length(users))
    weight_sum = zeros(eltype(weights), length(users))
    for u = 1:length(users)
        for (i, weight) in zip(items, weights)
            if isnonzero(R[users[u], i])
                predictions[u] += weight * polynomial_transform(R[users[u], i], λ[6:10])
                weight_sum[u] += abs(weight)
            end
        end
    end
    for u = 1:length(users)
        predictions[u] /= (weight_sum[u] + 1)
    end
    predictions
end;

In [7]:
function collaborative_filtering(training, inference, get_neighborhood, λ)
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        maximum(training.item),
    )

    preds = zeros(eltype(λ), length(inference.rating))
    @tprogress Threads.@threads for item in collect(Set(inference.item))
        mask = inference.item .== item 
        preds[mask] = make_prediction(item, inference.user[mask], R, get_neighborhood, λ)
    end

    preds
end;

In [8]:
@with_kw struct cf_params
    name::Any
    training_residuals::Any
    validation_residuals::Any
    neighborhood_type::Any
    S::Any # the similarity matrix
    K::Any # the neighborhood size
    λ::Vector{Float32}
end;

to_dict(x::T) where {T} = Dict(string(fn) => getfield(x, fn) for fn ∈ fieldnames(T));

## Item based CF

In [9]:
function get_residuals(split, residual_alphas)
    df = get_split(split, false)
    ratings = df.rating - read_alpha(residual_alphas, split, false).rating
    RatingsDataset(df.user, df.item, ratings)
end

function get_training(residual_alphas)
    get_residuals("training", residual_alphas)
end

function get_validation(residual_alphas)
    get_residuals("validation", residual_alphas)
end

function get_inference()
    splits = reduce(cat, [get_split(split, false) for split in all_raw_splits])
    RatingsDataset(
        user = splits.user,
        item = splits.item,
        rating = fill(0.0f0, length(splits.rating)),
    )
end;

In [10]:
function optimize_model(param)
    # unpack parameters
    redirect_logging("../../data/alphas/$(param.name)")
    training = get_training(param.training_residuals)
    validation = get_validation(param.validation_residuals)
    S = read_similarity_matrix(param.S)
    K = param.K
    neighborhood_types = Dict("abs" => get_abs_neighborhood)
    neighborhoods = i -> neighborhood_types[param.neighborhood_type](i, S, K)

    # optimize hyperparameters
    function validation_mse(λ)
        preds = collaborative_filtering(training, validation, neighborhoods, λ)
        loss = residualized_loss(param.validation_residuals, false, preds)
        @info "loss: $loss"
        loss
    end
    res = Optim.optimize(
        validation_mse,
        param.λ,
        Optim.NewtonTrustRegion(),
        autodiff = :forward,
        Optim.Options(show_trace = true, extended_trace = true),
    )
    param = @set param.λ = Optim.minimizer(res)

    # save predictions
    inference = get_inference()
    preds = collaborative_filtering(training, inference, neighborhoods, param.λ)
    sparse_preds = sparse(inference.user, inference.item, preds)
    function model(users, items, predictions)
        result = zeros(length(users))
        for i = 1:length(users)
            if users[i] <= size(predictions)[1] && items[i] <= size(predictions)[2]
                result[i] = predictions[users[i], items[i]]
            end
        end
        result
    end
    write_predictions(
        (users, items) -> model(users, items, sparse_preds),
        outdir = param.name,
        residual_alphas = param.validation_residuals,
    )
    write_params(to_dict(param), param.name)
end;